<a href="https://colab.research.google.com/github/valmirf/redes_neurais_lc/blob/main/MLP/Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [420]:
!git clone https://github.com/valmirf/redes_neurais_lc.git

fatal: destination path 'redes_neurais_lc' already exists and is not an empty directory.


##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation. 

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [421]:
import random
import numpy as np

A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [422]:
# Função de Ativação Sigmóide
def sigmoid(net):
    return 1.0/(1.0+np.exp(-net))

# Função para retornar as derivadas da função Sigmóide
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

def relu(net):
    ze = np.zeros(net.shape)
    return np.max([net, ze], axis=0)
    
def relu_prime(z):
    saida = np.zeros(z.shape)
    for i, j in zip(range(z.shape[0]), z):
        saida[i] = (0, 1)[bool(j>0)]
    return saida

# Classe Network
class Network(object):

    def __init__(self, sizes, func = sigmoid, ifunc = sigmoid_prime):
        self.num_layers = len(sizes)  #número de neurônios em cada camada
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] #limiar
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] #pesos
        self.func = func
        self.ifunc = ifunc

    def feedforward(self, x):
        """Retorna a saída da rede z se `x` for entrada."""
        for b, w in zip(self.biases, self.weights):
            x = self.func(np.dot(w, x)+b)
#             x = sigmoid(np.dot(w, x)+b) #net = (∑xw+b)
        return x

    def SGD(self, training_data, epochs, mini_batch_size, 𝜂, test_data=None):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente. 
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""

        #dataset de treino
        training_data = list(training_data)
        n = len(training_data)
        
        #dataset de teste
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            #técnica que realiza o treinamento por lotes
            #mini_batch_size = tamanho do lote
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, 𝜂)
            
            if test_data:
                acc = self.evaluate(test_data)
                print("Epoch {} : {} / {} = {}%".format(j,acc,n_test,(acc*100)/n_test));
   
            else:
                print("Epoch {} finalizada".format(j))

    def update_mini_batch(self, mini_batch, 𝜂):
        """Atualiza os pesos e limiares da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `a` é a taxa de aprendizado."""

        #inicializa matriz com derivadas de pesos e limiares
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        for x, y in mini_batch:
            #resultado dos deltas do backpropagation sem a multiplicação da taxa de aprendizagem
            #soma os deltas do minibatch
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            
        #atualiza pesos e limiares (𝜂*𝛿*f’(net)*𝑥)  
        self.weights = [w-(𝜂/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(𝜂/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo J_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        
        # Feedforward
        activation = x

        # Lista para armazenar todas as saídas dos neurônios (z), camada por camada
        activations = [x] 

        # Lista para armazenar todos os vetores net, camada por camada
        nets = [] 

        for b, w in zip(self.biases, self.weights):
            net = np.dot(w, activation)+b   
            nets.append(net)
            activation = self.func(net)
#             activation = sigmoid(net) #z = valor de saída do neurônio
            activations.append(activation)
        
        # Backward pass 
        
        #última camada -(u-z)f'(net)
        delta = self.cost_derivative(activations[-1], y) * self.func(nets[-1])
#         delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(nets[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) #(𝑦−𝑧)*f’(net)*𝑥
        
        # l = 1 significa a última camada de neurônios, l = 2 é a penúltima e assim por diante. 
        for l in range(2, self.num_layers):
            net = nets[-l]
            zs = self.ifunc(net)
#             zs = sigmoid_prime(net)
            #delta da camada intermediaria. Note que utiliza o delta calculado anteriormente
            delta = np.dot(self.weights[-l+1].transpose(), delta) * zs 
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose()) #∑(𝛿𝑤)f’(net)𝑥
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Retorna o número de entradas de teste para as quais a rede neural 
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""

        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)




Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [423]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('redes_neurais_lc/data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


#Executa a rede neural

Parâmetros de rede:
         2º param é contagem de épocas
         3º param é tamanho do lote
         4º param é a taxa de aprendizado (𝜂)




In [424]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)
test_data = list(test_data)

#arquitetura da rede
arquitecture = [784, 30, 10]
# mlp = Network(arquitecture)
# mlp.SGD(training_data, 10, 32, 0.3, test_data=test_data)


# **Mini-Projeto**
1) Realizar avaliações modificando os seguintes parâmetros:     

     a) Taxa de aprendizagem: 0.3, 0.4 e 0.5
     b) Função de ativação RELU (https://matheusfacure.github.io/2017/07/12/activ-func/)
     c) Rede com uma camada intermediária com 3 configurações diferentes (Explicite a configuração utilizada)
     d) Rede com duas camadas intermediárias com 3 configurações diferentes (Explicite a configuração utilizada)
     
      
Complete a Tabela abaixo com os resultados (Pra cada configuração de camadas intermediárias, execute as 3 taxas de aprendizagem pra função de ativação Sigmoide e Relu):


\begin{array}{|c|ccc|ccc|ccc|}\hline\\ \\
  1\;Camada\;Intermediária & & \mathcal{𝜂=0.3} & &  & \mathcal{𝜂=0.4} & & & \mathcal{𝜂=0.5} &  & \\ \hline 
Configurações & 1 & 2 & 3 & 1 & 2 & 3 & 1 & 2 & 3 & \\ \hline
Sigmoide  & 88.42\% & 89.81\% & 73.84\% & 89.9\% & 90.83\% & 76.05\% & 90.67\% & 83.4\% & 92.03\% & \\ \hline 
Relu  & 9.8\% & 9.8\% & 9.8\% & 9.8\% & 9.8\% & 9.8\% & 9.8\% & 9.8\% & 9.8\% & \\ \hline
  2\;Camadas\;Intermediárias & & \mathcal{𝜂=0.3} & &  & \mathcal{𝜂=0.4} & & & \mathcal{𝜂=0.5} &  & \\ \hline 
Configurações & 1 & 2 & 3 & 1 & 2 & 3 & 1 & 2 & 3 & \\ \hline 
Sigmoide  & 81.64\% & 83.96\% & 90.58\% & 91.55\% & 92.01\% & 92.29\% & 91.85\% & 92.64\% & 83.5\% & \\ \hline 
Relu  & 9.8\% & 9.8\% & 9.8\% & 9.8\% & 9.8\% & 9.8\% & 9.8\% & 9.8\% & 9.8\% & \\ \hline
\end{array}


In [425]:
#Questao 1, letra a

ml3 = Network(arquitecture)
ml4 = Network(arquitecture)
ml5 = Network(arquitecture)
print("Questao 1, letra A.\n")
print("Rede com 0.3 de taxa de aprendizado.")
ml3.SGD(training_data, 10, 32, 0.3, test_data=test_data)
print()
print("Rede com 0.4 de taxa de aprendizado.")
ml4.SGD(training_data, 10, 32, 0.4, test_data=test_data)
print()
print("Rede com 0.5 de taxa de aprendizado.")
ml5.SGD(training_data, 10, 32, 0.5, test_data=test_data)

Questao 1, letra A.

Rede com 0.3 de taxa de aprendizado.
Epoch 0 : 4959 / 10000 = 49.59%
Epoch 1 : 7048 / 10000 = 70.48%
Epoch 2 : 8098 / 10000 = 80.98%
Epoch 3 : 8433 / 10000 = 84.33%
Epoch 4 : 8687 / 10000 = 86.87%
Epoch 5 : 8806 / 10000 = 88.06%
Epoch 6 : 8886 / 10000 = 88.86%
Epoch 7 : 8946 / 10000 = 89.46%
Epoch 8 : 8997 / 10000 = 89.97%
Epoch 9 : 9027 / 10000 = 90.27%

Rede com 0.4 de taxa de aprendizado.
Epoch 0 : 5478 / 10000 = 54.78%
Epoch 1 : 7031 / 10000 = 70.31%
Epoch 2 : 7775 / 10000 = 77.75%
Epoch 3 : 8643 / 10000 = 86.43%
Epoch 4 : 8774 / 10000 = 87.74%
Epoch 5 : 8895 / 10000 = 88.95%
Epoch 6 : 8934 / 10000 = 89.34%
Epoch 7 : 8987 / 10000 = 89.87%
Epoch 8 : 9028 / 10000 = 90.28%
Epoch 9 : 9060 / 10000 = 90.6%

Rede com 0.5 de taxa de aprendizado.
Epoch 0 : 6440 / 10000 = 64.4%
Epoch 1 : 7471 / 10000 = 74.71%
Epoch 2 : 8471 / 10000 = 84.71%
Epoch 3 : 8816 / 10000 = 88.16%
Epoch 4 : 8949 / 10000 = 89.49%
Epoch 5 : 9021 / 10000 = 90.21%
Epoch 6 : 9084 / 10000 = 90.84%
Epoc

In [426]:
ml3 = Network(arquitecture, func=relu, ifunc=relu_prime)
# ml3 = Network(arquitecture, func=sigmoid, ifunc=sigmoid_prime)
ml4 = Network(arquitecture, func=relu, ifunc=relu_prime)
ml5 = Network(arquitecture, func=relu, ifunc=relu_prime)

# print(ml3.biases)

print("Questao 1, letra B\n")
print("Rede com 0.3 de taxa de aprendizado.")
ml3.SGD(training_data, 1, 32, 0.3, test_data=test_data)
print()
print("Rede com 0.4 de taxa de aprendizado.")
ml4.SGD(training_data, 1, 32, 0.4, test_data=test_data)
print()
print("Rede com 0.5 de taxa de aprendizado.")
ml5.SGD(training_data, 1, 32, 0.5, test_data=test_data)



Questao 1, letra B

Rede com 0.3 de taxa de aprendizado.
Epoch 0 : 980 / 10000 = 9.8%

Rede com 0.4 de taxa de aprendizado.
Epoch 0 : 980 / 10000 = 9.8%

Rede com 0.5 de taxa de aprendizado.
Epoch 0 : 980 / 10000 = 9.8%


In [427]:
ml3 = Network([784, 15, 10])
ml4 = Network([784, 30, 10])
ml5 = Network([784, 45, 10])

# print(ml3.biases)

print("Questao 1, letra C\n")
print("Rede com 15 neuronios na camada intermediaria.")
ml3.SGD(training_data, 10, 32, 0.4, test_data=test_data)
print()
print("Rede com 30 neuronios na camada intermediaria.")
ml4.SGD(training_data, 10, 32, 0.4, test_data=test_data)
print()
print("Rede com 45 neuronios na camada intermediaria.")
ml5.SGD(training_data, 10, 32, 0.4, test_data=test_data)

Questao 1, letra C

Rede com 15 neuronios na camada intermediaria.
Epoch 0 : 4468 / 10000 = 44.68%
Epoch 1 : 6816 / 10000 = 68.16%
Epoch 2 : 7424 / 10000 = 74.24%
Epoch 3 : 7698 / 10000 = 76.98%
Epoch 4 : 7846 / 10000 = 78.46%
Epoch 5 : 7911 / 10000 = 79.11%
Epoch 6 : 7970 / 10000 = 79.7%
Epoch 7 : 8042 / 10000 = 80.42%
Epoch 8 : 8090 / 10000 = 80.9%
Epoch 9 : 8086 / 10000 = 80.86%

Rede com 30 neuronios na camada intermediaria.
Epoch 0 : 4578 / 10000 = 45.78%
Epoch 1 : 6869 / 10000 = 68.69%
Epoch 2 : 8344 / 10000 = 83.44%
Epoch 3 : 8667 / 10000 = 86.67%
Epoch 4 : 8840 / 10000 = 88.4%
Epoch 5 : 8945 / 10000 = 89.45%
Epoch 6 : 9016 / 10000 = 90.16%
Epoch 7 : 9055 / 10000 = 90.55%
Epoch 8 : 9091 / 10000 = 90.91%
Epoch 9 : 9128 / 10000 = 91.28%

Rede com 45 neuronios na camada intermediaria.
Epoch 0 : 5110 / 10000 = 51.1%
Epoch 1 : 6415 / 10000 = 64.15%
Epoch 2 : 7096 / 10000 = 70.96%
Epoch 3 : 7260 / 10000 = 72.6%
Epoch 4 : 7337 / 10000 = 73.37%
Epoch 5 : 7379 / 10000 = 73.79%
Epoch 6 : 

In [428]:
ml3 = Network([784, 40, 20, 10])
ml4 = Network([784, 50, 20, 10])
ml5 = Network([784, 60, 20, 10])

# print(ml3.biases)

print("Questao 1, letra D\n")
print("Rede com [784, 40, 20, 10] neuronios na camada intermediaria.")
ml3.SGD(training_data, 10, 32, 0.4, test_data=test_data)
print()
print("Rede com [784, 50, 20, 10] neuronios na camada intermediaria.")
ml4.SGD(training_data, 10, 32, 0.4, test_data=test_data)
print()
print("Rede com [784, 60, 20, 10] neuronios na camada intermediaria.")
ml5.SGD(training_data, 10, 32, 0.4, test_data=test_data)

Questao 1, letra D

Rede com [784, 40, 20, 10] neuronios na camada intermediaria.
Epoch 0 : 5797 / 10000 = 57.97%
Epoch 1 : 8010 / 10000 = 80.1%
Epoch 2 : 8536 / 10000 = 85.36%
Epoch 3 : 8791 / 10000 = 87.91%
Epoch 4 : 8942 / 10000 = 89.42%
Epoch 5 : 9033 / 10000 = 90.33%
Epoch 6 : 9082 / 10000 = 90.82%
Epoch 7 : 9129 / 10000 = 91.29%
Epoch 8 : 9153 / 10000 = 91.53%
Epoch 9 : 9170 / 10000 = 91.7%

Rede com [784, 50, 20, 10] neuronios na camada intermediaria.
Epoch 0 : 5393 / 10000 = 53.93%
Epoch 1 : 7945 / 10000 = 79.45%
Epoch 2 : 8589 / 10000 = 85.89%
Epoch 3 : 8810 / 10000 = 88.1%
Epoch 4 : 8945 / 10000 = 89.45%
Epoch 5 : 9038 / 10000 = 90.38%
Epoch 6 : 9076 / 10000 = 90.76%
Epoch 7 : 9114 / 10000 = 91.14%
Epoch 8 : 9141 / 10000 = 91.41%
Epoch 9 : 9156 / 10000 = 91.56%

Rede com [784, 60, 20, 10] neuronios na camada intermediaria.
Epoch 0 : 6152 / 10000 = 61.52%
Epoch 1 : 7754 / 10000 = 77.54%
Epoch 2 : 8470 / 10000 = 84.7%
Epoch 3 : 8754 / 10000 = 87.54%
Epoch 4 : 8944 / 10000 = 89.

In [429]:
print("Rede [784, 15, 10] com 0.3 de aprendizado")
arede = Network([784, 15, 10])
arede.SGD(training_data, 10, 32, 0.3, test_data=test_data)
print("Rede [784, 30, 10] com 0.3 de aprendizado")
arede = Network([784, 30, 10])
arede.SGD(training_data, 10, 32, 0.3, test_data=test_data)
print("Rede [784, 45, 10] com 0.3 de aprendizado")
arede = Network([784, 45, 10])
arede.SGD(training_data, 10, 32, 0.3, test_data=test_data)

print("Rede [784, 15, 10] com 0.4 de aprendizado")
arede = Network([784, 15, 10])
arede.SGD(training_data, 10, 32, 0.4, test_data=test_data)
print("Rede [784, 30, 10] com 0.4 de aprendizado")
arede = Network([784, 30, 10])
arede.SGD(training_data, 10, 32, 0.4, test_data=test_data)
print("Rede [784, 45, 10] com 0.4 de aprendizado")
arede = Network([784, 45, 10])
arede.SGD(training_data, 10, 32, 0.4, test_data=test_data)

print("Rede [784, 15, 10] com 0.5 de aprendizado")
arede = Network([784, 15, 10])
arede.SGD(training_data, 10, 32, 0.5, test_data=test_data)
print("Rede [784, 30, 10] com 0.5 de aprendizado")
arede = Network([784, 30, 10])
arede.SGD(training_data, 10, 32, 0.5, test_data=test_data)
print("Rede [784, 45, 10] com 0.5 de aprendizado")
arede = Network([784, 45, 10])
arede.SGD(training_data, 10, 32, 0.5, test_data=test_data)

Rede [784, 15, 10] com 0.3 de aprendizado
Epoch 0 : 4593 / 10000 = 45.93%
Epoch 1 : 6420 / 10000 = 64.2%
Epoch 2 : 7527 / 10000 = 75.27%
Epoch 3 : 8115 / 10000 = 81.15%
Epoch 4 : 8404 / 10000 = 84.04%
Epoch 5 : 8555 / 10000 = 85.55%
Epoch 6 : 8628 / 10000 = 86.28%
Epoch 7 : 8721 / 10000 = 87.21%
Epoch 8 : 8793 / 10000 = 87.93%
Epoch 9 : 8842 / 10000 = 88.42%
Rede [784, 30, 10] com 0.3 de aprendizado
Epoch 0 : 5309 / 10000 = 53.09%
Epoch 1 : 7337 / 10000 = 73.37%
Epoch 2 : 7842 / 10000 = 78.42%
Epoch 3 : 8045 / 10000 = 80.45%
Epoch 4 : 8158 / 10000 = 81.58%
Epoch 5 : 8260 / 10000 = 82.6%
Epoch 6 : 8440 / 10000 = 84.4%
Epoch 7 : 8851 / 10000 = 88.51%
Epoch 8 : 8919 / 10000 = 89.19%
Epoch 9 : 8981 / 10000 = 89.81%
Rede [784, 45, 10] com 0.3 de aprendizado
Epoch 0 : 3890 / 10000 = 38.9%
Epoch 1 : 5405 / 10000 = 54.05%
Epoch 2 : 6050 / 10000 = 60.5%
Epoch 3 : 6289 / 10000 = 62.89%
Epoch 4 : 6367 / 10000 = 63.67%
Epoch 5 : 6410 / 10000 = 64.1%
Epoch 6 : 6448 / 10000 = 64.48%
Epoch 7 : 6495 /

In [430]:
print("Rede [784, 40, 20, 10] com 0.3 de aprendizado")
arede = Network([784, 40, 20, 10])
arede.SGD(training_data, 10, 32, 0.3, test_data=test_data)
print("Rede [784, 50, 20, 10] com 0.3 de aprendizado")
arede = Network([784, 50, 20, 10])
arede.SGD(training_data, 10, 32, 0.3, test_data=test_data)
print("Rede [784, 60, 20, 10] com 0.3 de aprendizado")
arede = Network([784, 60, 20, 10])
arede.SGD(training_data, 10, 32, 0.3, test_data=test_data)

print("Rede [784, 40, 20, 10] com 0.4 de aprendizado")
arede = Network([784, 40, 20, 10])
arede.SGD(training_data, 10, 32, 0.4, test_data=test_data)
print("Rede [784, 50, 20, 10] com 0.4 de aprendizado")
arede = Network([784, 50, 20, 10])
arede.SGD(training_data, 10, 32, 0.4, test_data=test_data)
print("Rede [784, 60, 20, 10] com 0.4 de aprendizado")
arede = Network([784, 60, 20, 10])
arede.SGD(training_data, 10, 32, 0.4, test_data=test_data)

print("Rede [784, 40, 20, 10] com 0.5 de aprendizado")
arede = Network([784, 40, 20, 10])
arede.SGD(training_data, 10, 32, 0.5, test_data=test_data)
print("Rede [784, 50, 20, 10] com 0.5 de aprendizado")
arede = Network([784, 50, 20, 10])
arede.SGD(training_data, 10, 32, 0.5, test_data=test_data)
print("Rede [784, 60, 20, 10] com 0.5 de aprendizado")
arede = Network([784, 60, 20, 10])
arede.SGD(training_data, 10, 32, 0.5, test_data=test_data)

Rede [784, 40, 20, 10] com 0.3 de aprendizado
Epoch 0 : 5430 / 10000 = 54.3%
Epoch 1 : 7057 / 10000 = 70.57%
Epoch 2 : 7517 / 10000 = 75.17%
Epoch 3 : 7778 / 10000 = 77.78%
Epoch 4 : 7917 / 10000 = 79.17%
Epoch 5 : 7995 / 10000 = 79.95%
Epoch 6 : 8065 / 10000 = 80.65%
Epoch 7 : 8108 / 10000 = 81.08%
Epoch 8 : 8152 / 10000 = 81.52%
Epoch 9 : 8164 / 10000 = 81.64%
Rede [784, 50, 20, 10] com 0.3 de aprendizado
Epoch 0 : 4907 / 10000 = 49.07%
Epoch 1 : 6658 / 10000 = 66.58%
Epoch 2 : 7576 / 10000 = 75.76%
Epoch 3 : 7919 / 10000 = 79.19%
Epoch 4 : 8078 / 10000 = 80.78%
Epoch 5 : 8189 / 10000 = 81.89%
Epoch 6 : 8268 / 10000 = 82.68%
Epoch 7 : 8324 / 10000 = 83.24%
Epoch 8 : 8360 / 10000 = 83.6%
Epoch 9 : 8396 / 10000 = 83.96%
Rede [784, 60, 20, 10] com 0.3 de aprendizado
Epoch 0 : 5243 / 10000 = 52.43%
Epoch 1 : 6868 / 10000 = 68.68%
Epoch 2 : 7883 / 10000 = 78.83%
Epoch 3 : 8426 / 10000 = 84.26%
Epoch 4 : 8644 / 10000 = 86.44%
Epoch 5 : 8815 / 10000 = 88.15%
Epoch 6 : 8894 / 10000 = 88.94%
